# Projekat 2
- Rok za predaju je termin odbrane u toku 2. kolokvijumske nedelje
- Projekat je moguće raditi u paru ili samostalno
- Svaka stavka nosi po 10 bodova

Koristeći PyCuda okurženje napisati CUDA program za (matrično) množenje matrica.

1. Program koji vrši množenja matrica malih dimenzija (množenje se može izvršiti upotrebom jednom bloka niti)
2. Probram koji vrši množenje matrica većih dimenzija, upotrebom većeg broja CUDA blokova.
3. Ubrzati rešenje iz stavke 2 upotrebom deljene memorije (tako da niti jednog bloka prvo dovuku deo podataka u deljenu memeorju, a potom sve čitaju iz deljene memorije)
4. Izmeniti rešenje iz tačke 3 tako da se pri množenju druga matrica transponuje ($Rezultat = A \cdot B^T$)

In [0]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 44.6MB/s 
     |████████████████████████████████| 61kB 10.9MB/s 
     |████████████████████████████████| 471kB 50.7MB/s 
  Created wheel for pycuda: filename=pycuda-2019.1.2-cp36-cp36m-linux_x86_64.whl size=4536631 sha256=38ce925e7746c8be055ee3be66c741b96f362768bca4f81c4f9283c3135b8538
  Stored in directory: /root/.cache/pip/wheels/a6/60/f0/b1c430c73d281ac3e46070480db50f7907364eb6f6d3188396
  Created wheel for pytools: filename=pytools-2019.1.1-py2.py3-none-any.whl size=58424 sha256=57eed7afa606b28999baafcbf190a01da60de508ecf38462a0f2cb5a293ef33d
  Stored in directory: /root/.cache/pip/wheels/83/df/0b/75ac4572aaa93e3eba6a58472635d0fda907f5f4cf884a3a0c
  Created wheel for mako: filename=Mako-1.1.0-cp36-none-any.whl size=75363 sha256=816246a1416f287a39afab4bb0a04f9c034a862b1a5f2013720d4838614cbede
  Stored in directory: /root/.cache/pip/wheels/98/32/7b/a291926643fc1d1e02593e0d9e247c5a866a366b8343b7aa27
Successfully built pycuda pytools mako


In [0]:
!nvidia-smi

Thu Dec 19 16:52:04 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

1. Program koji vrši množenja matrica malih dimenzija (množenje se može izvršiti upotrebom jednom bloka niti)

In [0]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

mod = SourceModule(""" 
        __global__ void multiplyMatrixOneBlock(float *a,float *b, float *c, int colsA,int colsB){
              int idxC = threadIdx.x+threadIdx.y*colsB;
              int idxA = threadIdx.y*colsA;
              int idxB = threadIdx.x;

              int i=0;

              for(i;i<colsA;i++){
                c[idxC] += a[idxA+i]*b[idxB];
                idxB += colsB;
              }
            }
        """)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu(10): warning: expression has no effect




In [0]:
import numpy as np

a = np.random.randn(20,10).astype(dtype=np.float32)
b = np.random.randn(10,10).astype(dtype=np.float32)
# print(b)
res = a.dot(b)

c = np.zeros_like(res)
# print(c)

a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu,a)

b_gpu = cuda.mem_alloc(b.nbytes)
cuda.memcpy_htod(b_gpu,b)

c_gpu = cuda.mem_alloc(c.nbytes)
cuda.memcpy_htod(c_gpu,c)

func = mod.get_function("multiplyMatrixOneBlock")
func(a_gpu,b_gpu,c_gpu,np.int32(a.shape[1]),np.int32(b.shape[1]),block=(np.int(b.shape[1]),np.int(a.shape[0]),1),grid=(1,1,1))

cuda.memcpy_dtoh(c,c_gpu)

print(res)
print("\n")
print(c)

print("Dobijen ocekivan rezultat: ",(c==res).all())


[[ 6.3911610e+00  1.0221982e+00 -8.5005313e-02  1.9880366e+00
  -6.4945817e-01  1.9238430e+00 -2.1429772e+00 -1.0079645e+00
   1.1161232e+00  4.2055637e-01]
 [ 1.7334768e-01  6.1608270e-02 -2.4337206e+00  1.1337092e-01
   2.6067014e+00 -1.9993405e+00 -1.5743302e+00  8.1213498e-01
   3.9028460e-01 -8.2406896e-01]
 [-4.1025046e-02 -2.3925798e+00 -1.4787147e+00  2.0284991e+00
  -2.3220094e-01 -7.4490100e-01  1.1805792e+00  2.3373911e+00
  -3.2321701e+00 -3.0012908e+00]
 [-2.4612210e+00  1.1300583e+00  1.0958425e+00 -4.2690164e-01
   8.7957931e-01  1.3533710e+00 -4.6909055e-01 -1.2978941e-01
   4.2880520e-01  4.6252719e-01]
 [-9.7801048e-01 -1.0025293e+00 -2.3281341e+00 -1.7007207e+00
   3.9787742e-01  9.9198765e-01 -2.6226246e+00  3.0047281e+00
  -4.2363439e-02 -7.7633488e-01]
 [-2.5449617e+00 -1.3945469e+00 -9.0498984e-01 -1.0156323e+00
  -1.4662622e+00 -1.2191536e+00 -4.5226914e-01  8.5056275e-01
   1.7681444e+00 -6.9471769e-02]
 [ 1.6800536e+00  1.7488817e+00  2.6348259e+00  2.2133408e

2. Program koji vrši množenje matrica većih dimenzija, upotrebom većeg broja CUDA blokova.

In [0]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

mod = SourceModule(""" 
        __global__ void multiplyMatrixMultipleBlocksaaa(float *a,float *b, float *c,int colsB,int rowsA,int colsA){
              long idxC = threadIdx.x + blockDim.x*blockIdx.x + threadIdx.y*colsB + blockDim.y*blockIdx.y*colsB;
              long idxA = (threadIdx.y + blockDim.y*blockIdx.y)*colsA;
              long idxB = blockDim.x*blockIdx.x+threadIdx.x;

              if(idxB > colsB) return;
              if(threadIdx.y + blockDim.y*blockIdx.y > rowsA) return;
             

              for(int i = 0;i<colsA;i++){
                c[idxC] += a[idxA+i]*b[idxB];
                idxB += colsB;
              }
            }
        """)

In [0]:
import numpy as np
import math

a = np.random.randn(90,64).astype(dtype=np.float32)
b = np.random.randn(64,50).astype(dtype=np.float32)

res = a.dot(b)

c = np.zeros_like(res)

a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu,a)

b_gpu = cuda.mem_alloc(b.nbytes)
cuda.memcpy_htod(b_gpu,b)

c_gpu = cuda.mem_alloc(c.nbytes)
cuda.memcpy_htod(c_gpu,c)

func = mod.get_function("multiplyMatrixMultipleBlocksaaa")
func(a_gpu, b_gpu, c_gpu,np.int32(b.shape[1]),np.int32(a.shape[0]),np.int32(a.shape[1]),block=(32, 32, 1), grid=(math.ceil(b.shape[1]/32), math.ceil(a.shape[0]/32), 1))

cuda.memcpy_dtoh(c,c_gpu)
print(res)
print("\n")
print(c)
print("Dobijen ocekivan rezultat: ",(c==res).all())

[[  7.3198705   -4.542291     1.3440619  ...  -7.3176413    7.25597
   -3.1695373 ]
 [  4.0502343   -6.9215407    9.709128   ...   9.708631     2.256934
   -1.6978451 ]
 [ -2.4553452   -5.312149     0.52720547 ...   6.414436   -13.825711
   -2.2888393 ]
 ...
 [  8.257005    -7.0827      -0.5585329  ...  -2.1756194   -2.876888
    7.0910373 ]
 [  8.512214    -2.5961447   -1.1105841  ...  -3.6714163    7.7299633
   -4.9011827 ]
 [ -7.176016    -8.833031    11.078852   ...  -3.1767344   -6.027856
    1.0417136 ]]


[[  7.3198705   -4.542291     1.3440619  ...  -7.3176413    7.25597
   -3.1695373 ]
 [  4.0502343   -6.9215407    9.709128   ...   9.708631     2.256934
   -1.6978451 ]
 [ -2.4553452   -5.312149     0.52720547 ...   6.414436   -13.825711
   -2.2888393 ]
 ...
 [  8.257005    -7.0827      -0.5585329  ...  -2.1756194   -2.876888
    7.0910373 ]
 [  8.512214    -2.5961447   -1.1105841  ...  -3.6714163    7.7299633
   -4.9011827 ]
 [ -7.176016    -8.833031    11.078852   ...  -3.176

3. Ubrzati rešenje iz stavke 2 upotrebom deljene memorije (tako da niti jednog bloka prvo dovuku deo podataka u deljenu memeorju, a potom sve čitaju iz deljene memorije)    

In [0]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

mod = SourceModule(""" 
        __global__ void multiplyMatrixMultipleBlocksSharedMemory(float *a,float *b, float *c,int rowsA, int colsA,
                                                                  int rowsB,int colsB,int rowsC,int colsC){
              float cValue = 0;

              int row = blockIdx.y * blockDim.y + threadIdx.y;
              int col = blockIdx.x * blockDim.y + threadIdx.x;

              __shared__ float aVal[32][32];
              __shared__ float bVal[32][32];

              for(int i = 0; i < (blockDim.y + colsA - 1)/blockDim.y;i++){
                if(i*blockDim.y + threadIdx.x < colsA && row < rowsA){
                  aVal[threadIdx.y][threadIdx.x] = a[row*colsA + i*blockDim.y + threadIdx.x];
                } else {
                  aVal[threadIdx.y][threadIdx.x] = 0.0;
                }

                if(i * blockDim.y + threadIdx.y < rowsB && col < colsB){
                  bVal[threadIdx.y][threadIdx.x] = b[(i*blockDim.y + threadIdx.y)* colsB + col];
                } else {
                  bVal[threadIdx.y][threadIdx.x] = 0.0;
                }

                __syncthreads();

                for(int j = 0 ; j < blockDim.y ; ++j){
                  cValue += aVal[threadIdx.y][j] * bVal[j][threadIdx.x];
                }

                __syncthreads();

                if(row < rowsC && col < colsC){
                  c[((blockIdx.y * blockDim.y) + threadIdx.y) * colsC + (blockIdx.x * blockDim.x) + threadIdx.x] = cValue;
                }                
              } 
            }
        """)

In [0]:
import numpy as np
import math

a = np.random.randn(40,64).astype(dtype=np.float32)
b = np.random.randn(64,90).astype(dtype=np.float32)

res = a.dot(b)

c = np.zeros_like(res)

a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu,a)

b_gpu = cuda.mem_alloc(b.nbytes)
cuda.memcpy_htod(b_gpu,b)

c_gpu = cuda.mem_alloc(c.nbytes)
cuda.memcpy_htod(c_gpu,c)

func = mod.get_function("multiplyMatrixMultipleBlocksSharedMemory")
func(a_gpu, b_gpu, c_gpu,np.int32(a.shape[0]),np.int32(a.shape[1]),np.int32(b.shape[0]),np.int32(b.shape[1]),
     np.int32(a.shape[0]),np.int32(b.shape[1]),block=(32, 32, 1), grid=(math.ceil(b.shape[1]/32), math.ceil(a.shape[0]/32), 1))

cuda.memcpy_dtoh(c,c_gpu)
print(res)
print("\n")
print(c)
print("Dobijen ocekivan rezultat: ",(c==res).all())

[[  7.6361303   -4.492075     0.95769763 ...   4.324452    -7.24102
    2.2083914 ]
 [ -2.6228824  -16.324022    -3.7561815  ...   1.7153242   -2.4590335
    8.217871  ]
 [ -2.7524567    2.3109167   10.139936   ...   1.5209172   -0.6137072
   -9.850053  ]
 ...
 [ -1.426733   -10.932176     9.833535   ...  -9.744608    -9.171083
   -5.959076  ]
 [  3.2013073  -21.283293   -11.389342   ... -17.901306    -4.1969113
    3.023788  ]
 [ -1.0582824    4.341552     5.0392947  ...   3.5763683   -5.9991574
   -3.5220878 ]]


[[  7.6361303   -4.492075     0.95769763 ...   4.324452    -7.24102
    2.2083914 ]
 [ -2.6228824  -16.324022    -3.7561815  ...   1.7153242   -2.4590335
    8.217871  ]
 [ -2.7524567    2.3109167   10.139936   ...   1.5209172   -0.6137072
   -9.850053  ]
 ...
 [ -1.426733   -10.932176     9.833535   ...  -9.744608    -9.171083
   -5.959076  ]
 [  3.2013073  -21.283293   -11.389342   ... -17.901306    -4.1969113
    3.023788  ]
 [ -1.0582824    4.341552     5.0392947  ...   

4. Izmeniti rešenje iz tačke 3 tako da se pri množenju druga matrica transponuje ($Rezultat = A \cdot B^T$)